## 2-4　集約IDに基づくサンプリング

In [1]:
import pandas as pd
import psycopg2
import config

In [2]:
connection = psycopg2.connect(**config.connection_config)

In [3]:
query ="""

SELECT
  COUNT(*)
FROM
  reserve_tb
;

"""

pd.read_sql(query, con=connection)

,count
0,4030


In [4]:
query ="""

SELECT
  *,
  FIRST_VALUE(RANDOM()) OVER (PARTITION BY customer_id) AS random_num
FROM
  reserve_tb
LIMIT 5
;

"""

pd.read_sql(query, con=connection)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,random_num
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,0.017267
1,r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,0.081416
2,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.313030
3,r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,0.818811
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.003505


In [5]:
query ="""

WITH reserve_tb_random AS(
  SELECT
    *,
    FIRST_VALUE(RANDOM()) OVER (PARTITION BY customer_id) AS random_num
  FROM
    reserve_tb
)

SELECT
  *
FROM
  reserve_tb_random
WHERE
  random_num <= 0.5
;

"""

pd.read_sql(query, con=connection)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,random_num
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,0.480138
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.349349
2,r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,0.493735
3,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.036152
4,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0.055022
...,...,...,...,...,...,...,...,...,...,...
2056,r4019,h_224,c_996,2017-12-10 19:39:53,2017-12-20,11:30:00,2017-12-23,3,36000,0.100696
2057,r4015,h_225,c_996,2016-12-03 21:40:17,2016-12-22,11:30:00,2016-12-23,3,21600,0.234111
2058,r4020,h_243,c_996,2018-04-22 20:41:29,2018-05-21,12:00:00,2018-05-22,2,55400,0.373126
2059,r4023,h_4,c_999,2016-04-21 21:50:14,2016-04-22,12:00:00,2016-04-25,4,62400,0.127952
